## 1. 安装package openai
    ! pip install openai

In [1]:
import openai 
print("version =",openai.__version__)

version = 1.12.0


In [2]:
api_key_str = "sk-0g56v3qYextZ5b1JcxksAO6ZxJ3ZP7Y9uEVT4nEc6HRkVYde"

## 2. chat.completions

In [3]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)
 
completion = client.chat.completions.create(
  model="moonshot-v1-8k",
  messages=[
    {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
    {"role": "user", "content": "你好，我叫李雷，1+1等于多少？"}
  ],
  temperature=0.3,
)
 
print(completion.choices[0].message)

ChatCompletionMessage(content='你好，李雷！1+1等于2。如果你有更复杂的数学问题或者其他问题，也可以随时问我。', role='assistant', function_call=None, tool_calls=None)


In [5]:
print(completion.choices[0].message.content)

你好，李雷！1+1等于2。如果你有更复杂的数学问题或者其他问题，也可以随时问我。


## 3. 获取可用的模型列表

In [6]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)
 
history = [
    {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"}
]
 
def chat(query, history):
    history += [{
        "role": "user", 
        "content": query
    }]
    completion = client.chat.completions.create(
        model="moonshot-v1-8k",
        messages=history,
        temperature=0.3,
    )
    result = completion.choices[0].message.content
    history += [{
        "role": "assistant",
        "content": result
    }]
    return result
 
print(chat("地球的自转周期是多少？", history))

地球的自转周期，也就是地球绕自己的轴旋转一周所需的时间，通常被称为一天。这个周期大约是24小时。不过，由于地球的自转速度并不是完全均匀的，会有极微小的变化，因此实际上地球的自转周期可以略微短于或长于24小时，这种现象称为地球时。此外，由于地球在自转的同时还绕太阳公转，所以太阳日（太阳回到同样位置所需的时间）会比恒星日（地球自转一周的实际时间）略长一些。太阳日的平均长度大约是24小时整。


In [7]:
print(chat("月球呢？", history))

月球的自转周期与它的公转周期相同，这种现象称为潮汐锁定。月球的自转周期大约是27.3天，这也是它绕地球公转一周的时间。这意味着月球总是以同一面朝向地球，我们从地球上看到的月球表面只有一面，另一面通常被称为月球的背面或暗面。

需要注意的是，尽管月球的自转周期和公转周期相同，但由于地球也在绕太阳公转，所以月球需要额外的时间来完成一个完整的相位周期（从一个新月到下一个新月），这个周期大约是29.5天，称为朔望月。


In [ ]:
import os
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)
 
model_list = client.models.list()
model_data = model_list.data
 
for i, model in enumerate(model_data):
    print(f"model[{i}]:", model.id)

model[0]: moonshot-v1-8k
model[1]: moonshot-v1-32k
model[2]: moonshot-v1-128k


## 4. 文本分析任务


### （1） 从文本中抽取实体和关系：

今天我向大家介绍一下一个人。他是张丰毅1956年9月1日出生于河南省南阳市唐河县，1982年毕业于北京电影学院，是中国电影协会理事。1993年，与其他演员主演电影《霸王别姬》。 

待抽取信息：演员#出生年月#出生地#毕业院校


In [9]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)
 
completion = client.chat.completions.create(
  model="moonshot-v1-8k",
  messages=[
    {"role": "system", "content": "你是文本分析专家，请理解给定的文本，从中抽取演员姓名，及其出生年月、出生地、毕业院校。"},
    {"role": "user", "content": "给定文本：```今天我向大家介绍一下一个人。他是张丰毅1956年9月1日出生于河南省南阳市唐河县，1982年毕业于北京电影学院，是中国电影协会理事。1993年，与其他演员主演电影《霸王别姬》。```"}
  ],
  temperature=0.3,
)
 
print(completion.choices[0].message.content)

从给定的文本中，我们可以抽取以下信息：

演员姓名：张丰毅
出生年月：1956年9月1日
出生地：河南省南阳市唐河县
毕业院校：北京电影学院


### （2）命名实体识别

原文：据新华社电广东省清远市清城区政府昨日对外发布信息称，日前被实名举报涉嫌勒索企业、说“分分钟可以搞垮一间厂”的清城区环保局局长陈柏，已被免去清城区区委委员。

待抽取实体类型：机构名，人名，职位

In [10]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)
 
completion = client.chat.completions.create(
  model="moonshot-v1-8k",
  messages=[
    {"role": "system", "content": "你是文本分析专家，请理解给定的文本，从中抽取机构名、人名和职位。请以JSON形式给出抽取结果。"},
    {"role": "user", "content": "给定文本：```据新华社电广东省清远市清城区政府昨日对外发布信息称，日前被实名举报涉嫌勒索企业、说“分分钟可以搞垮一间厂”的清城区环保局局长陈柏，已被免去清城区区委委员。```"}
  ],
  temperature=0.3,
)
 
print(completion.choices[0].message.content)

```json
{
  "机构名": ["清远市清城区政府", "清城区环保局"],
  "人名": ["陈柏"],
  "职位": ["局长", "清城区区委委员"]
}
```


## （3）自定义待抽取信息

我今天胃不舒服，手有点麻木，浑身不舒服，前几天去医院医生说我有胃炎，需要做胃镜检查。

问题：症状#检查#身体部位#疾病

In [12]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)

prompt_str = "你是医疗专家，请理解给定的文本，从中抽取疾病，疾病症状、身体部位以及检查项目。请以JSON形式给出抽取结果。"
given_text = "我今天胃不舒服，手有点麻木，浑身不舒服，前几天去医院医生说我有胃炎，需要做胃镜检查。" 
completion = client.chat.completions.create(
  model="moonshot-v1-8k",
  messages=[
    {"role": "system", "content": prompt_str},
    {"role": "user", "content": "给定文本：" + given_text}
  ],
  temperature=0.3,
)
 
print(completion.choices[0].message.content)

{
  "疾病": [
    {
      "名称": "胃炎",
      "症状": [
        "胃不舒服",
        "浑身不舒服"
      ],
      "身体部位": [
        "胃"
      ],
      "检查项目": [
        "胃镜检查"
      ]
    }
  ]
}


### （4）关键词抽取


In [13]:
given_text = '''
人类正站在技术变革爆发点，Exascale超级电脑每秒进行百亿亿次级计算，AI发现的晶体结构数量是科学史上发现的45倍以上，仅用30天就研发出潜在抗癌新药，乌拉圭工业机器人数量比人口还多......
美国银行在3月21日的深度报告中指出，2024年将是“AI赋能一切”的一年，AI和其他技术发展之间形成一个巨大的正反馈，包括人工智能、计算、机器人、通信、医疗保健、能源等30项技术领域或将迎来突破。
而实现突破的技术商业化十分关键，美银预计相关市场规模约为16万亿美元。
从历史来看，技术进步导致财富集中和企业快速更迭，过去100年里3%的公司几乎创造了全球所有的净财富；自从2015年以来，大约三分之一标普500成分股被替换，技术加速市场变革和颠覆。
自去年来AI科技革命拉开帷幕，正带动各行各业加速发展，未来将更多取决于AI模型的落地和应用，美银指出了端侧AI设备、增强模拟、知识图谱、超算（HDC）、通用人工智能（AGI）等五大落地方向。
图片AI“接管”一切 发展形成“正反馈”
美银指出，人工智能发展正处于一个转折点，其正在以指数级的速度改变各行业，AI未来的发展将取决于以下三方面因素：
1、技术“交叉”：不同技术之间的相互促进，例如AI推动计算和通信技术的发展；
2、技术“稀缺”：在一个对技术的需求超过供应的世界，算力需求增长速度超过了摩尔定律的预测，数据和算力的稀缺性将成为挑战；
3、技术“经济性”：技术自身向着更低的成本和更高的回报发展。
进一步来看，美银认为，2024年将是“AI赋能一切”的一年，AI和其他技术发展之间形成一个巨大的正反馈。
AI连接并赋能技术、商业和社会，推动着技术奇点的发展。这一进程中，计算、通信和技术的发展为AI革命带来动力，反过来又形成了正向反馈循环，提供了更多的计算能力、通信资源和数据，进一步加速了AI进步。
美银还谈到了算力的重要性，如此多的数据，但算力却跟不上相应增长。
计算能力的需求增长速度超过了摩尔定律的预测，每两年增长275倍。
我们即将从每天生成百亿亿字节的数据——转向百万的六乘方字节，可再生能源超过80%的新产能也需要新的基础设施和材料，而这些原材料供应短缺。
此外，美银还提到，未来成本将进一步降低，所有这自动化、人工智能和技术上的投资都在全面降低价格并提高回报。例如，尽管过去20年内存驱动器的容量增加了超过20000倍，但每千兆字节的价格却下降了超过99%。
图片AI落地的五大应用领域
去年是“AI元年”，AI革命将从开始加速，2023年我们见证了生成式AI投资激增，自ChatGPT发布以来，已经引入了各种闭源和开源模型，各公司开始开发、采用或将AI集成到产品或业务中。
美银指出，创新步伐将从这里加速，更多的AI工具和应用可能很快会推出。这可能会在数字领域之外，为终端设备、机器人和生命科学的物理领域带来丰富的机会。
1、端侧AI设备：在本地设备（如智能手机、汽车、可穿戴设备）上部署AI功能/模型，有助于减少延迟、成本，有助于分担大型服务器的功率负载，提高整个AI生态系统的性能。
2、增强模拟：AI用于加速发现过程，识别最可行的模拟，加速新分子的创造，并降低成本，在物理世界中需要10年才能完成，现在可以在几周到几个月内完成这项任务，应用领域包括药物发现、芯片、化学品、材料。
3、知识图谱：知识图谱组织来自多个源的数据，捕获有关感兴趣主题的信息，并在它们之间建立联系。它们是解决LLM“幻觉”问题（即提供带有高度信心的不准确信息）并提高神经网络能力的关键。大多数组织中的数据专业人员通常花费25-30%的时间寻找和搜索相关数据。
4、超维度计算（HDC）：HDC使用高维向量来表示信息，而不是传统的二进制系统。它可以捕获更复杂的数据模式，并允许计算机保留更多记忆，从而减少计算和能源需求。HDC相比今天芯片中使用的技术，可以实现超过60%的能源节省。
5、通用人工智能（AGI）：AGI作为人工助理，它将具备在广泛任务上达到或超过人类水平的一般认知能力，能够实现自我学习，并且能够解决未被预先编程的任务。随着数据的增长、计算能力的提高和技术的创新，AI可能在未来十年内达到1500的智商，是人类平均智商的18倍。
美银补充称，尽管AI带来了许多好处，但也存在挑战，如端侧设备AI的功耗、成本、算法/软件优化和安全性问题。整体而言，美银报告预测，到2030年，AI可能会为全球增加15%-20%左右的经济价值。
'''

In [14]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)

prompt_str = "你是文本分析专家，请理解给定的文本，抽取关键词，以JSON形式输出。"

completion = client.chat.completions.create(
  model="moonshot-v1-8k",
  messages=[
    {"role": "system", "content": prompt_str},
    {"role": "user", "content": "给定文本：" + given_text}
  ],
  temperature=0.3,
)
 
print(completion.choices[0].message.content)

```json
{
  "keywords": [
    "技术变革",
    "Exascale超级电脑",
    "AI",
    "晶体结构",
    "抗癌新药",
    "乌拉圭工业机器人",
    "美国银行",
    "AI赋能",
    "正反馈",
    "技术领域",
    "商业化",
    "市场规模",
    "财富集中",
    "企业更迭",
    "标普500成分股",
    "AI科技革命",
    "AI模型",
    "端侧AI设备",
    "增强模拟",
    "知识图谱",
    "超算",
    "通用人工智能",
    "技术交叉",
    "技术稀缺",
    "技术经济性",
    "计算能力",
    "摩尔定律",
    "数据",
    "算力",
    "可再生能源",
    "原材料供应",
    "自动化",
    "投资",
    "价格",
    "回报",
    "AI落地应用",
    "数字领域",
    "物理领域",
    "药物发现",
    "芯片",
    "化学品",
    "材料",
    "LLM幻觉问题",
    "神经网络",
    "高维向量",
    "能源节省",
    "智商",
    "经济价值"
  ]
}
```


### (5) 文本摘要

In [15]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)

prompt_str = "你是文本分析专家，请理解给定的文本，给出文本的摘要。"

completion = client.chat.completions.create(
  model="moonshot-v1-8k",
  messages=[
    {"role": "system", "content": prompt_str},
    {"role": "user", "content": "给定文本：" + given_text}
  ],
  temperature=0.3,
)
 
print(completion.choices[0].message.content)

随着技术进步，人类正站在一个技术变革的爆发点。Exascale超级电脑、AI发现的晶体结构数量、抗癌新药的研发以及乌拉圭工业机器人数量的增加都是这一变革的例证。美国银行在3月21日的深度报告中预测，2024年将是AI赋能一切的一年，AI和其他技术发展之间将形成一个巨大的正反馈，包括人工智能、计算、机器人、通信、医疗保健、能源等30项技术领域将迎来突破。实现突破的技术商业化是关键，相关市场规模约为16万亿美元。技术进步导致财富集中和企业快速更迭，过去100年里3%的公司几乎创造了全球所有的净财富；自2015年以来，大约三分之一标普500成分股被替换。AI科技革命正带动各行各业加速发展，未来将更多取决于AI模型的落地和应用。美银指出了端侧AI设备、增强模拟、知识图谱、超算（HDC）、通用人工智能（AGI）等五大落地方向。尽管AI带来了许多好处，但也存在挑战，如端侧设备AI的功耗、成本、算法/软件优化和安全性问题。到2030年，AI可能会为全球增加15%-20%左右的经济价值。


### （6）文本分类

文本分类：大模型将在5年内创造10万亿美元的经济价值。

选项：文学，艺术，体育，健康，财经，天气

In [17]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)

prompt_str = "你是文本分析专家，请理解给定的文本，判断文本所属的类别，可选的类别范围是：文学，艺术，体育，健康，财经，天气。"

completion = client.chat.completions.create(
  model="moonshot-v1-8k",
  messages=[
    {"role": "system", "content": prompt_str},
    {"role": "user", "content": "给定文本：" + given_text}
  ],
  temperature=0.3,
)
 
print(completion.choices[0].message.content)

财经


### （7）情感分析

情感分析：这个看上去还可以，但其实我不喜欢。。

选项：积极，消极

In [18]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)

prompt_str = "你是文本分析专家，请理解给定的文本，判断文本的情感类别，可选的类别范围是：积极和消极。"
given_text = "这个看上去还可以，但其实我不喜欢。。"
completion = client.chat.completions.create(
  model="moonshot-v1-8k",
  messages=[
    {"role": "system", "content": prompt_str},
    {"role": "user", "content": "给定文本：" + given_text}
  ],
  temperature=0.3,
)
 
print(completion.choices[0].message.content)

消极


文字中包含了怎样的情感：超可爱的帅哥，爱了。。。

选项：厌恶，喜欢，开心，悲伤，惊讶，生气，害怕

In [19]:
from openai import OpenAI
 
client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)

def chat_completion(prompt_str, given_text):
    completion = client.chat.completions.create(
        model="moonshot-v1-8k",
        messages=[
            {"role": "system", "content": prompt_str},
            {"role": "user", "content": "给定文本：" + given_text}
        ],
        temperature=0.3,
    )
    return completion.choices[0].message.content

prompt_str = "你是文本分析专家，请理解给定的文本，判断文本的情感类别，可选的类别范围是：厌恶，喜欢，开心，悲伤，惊讶，生气，害怕。"
given_text = "超可爱的帅哥，爱了。。。"

print(chat_completion(prompt_str, given_text))

喜欢


方面级情感分析：


In [20]:
given_text = '''和老公的朋友们聚餐，选择了这家烧肉屋，据说人气很旺，于是定座的，我们坐在楼上，阁楼感觉有点空气不流通，我这个孕妇貌似不适合油烟味大的地方，虽然已经过了孕吐期，但一开始不太习惯，发现窗没有可以开的，都是固定的有点小失望，后来慢慢就习惯了，开始点了几个甜筒，很大一个，味道还凑合，后来大家点了很多肉，其中有一个肉味道很好，但是只顾着吃也忘记叫啥了，点了两份石锅饭也全干掉了，朋友的老婆点了冷面，第一次吃这样的，感觉不错啊！'''
prompt_str = '''作为专业的文本数据分析专家，请针对给定的评论给出方面级的情感分析判别，从18个方面给出情感效价，18个方面分别是：交通方便、位于商圈附近、是否容易寻找、排队时间、服务人员态度、停车方便、点菜/上菜速度、价格水平、性价比、折扣力度、装修、噪杂情况、就餐空间、卫生情况、菜品分量、菜品口味、菜品卖相、菜品推荐程度；以上每个方面均有四种情绪类别：1代表正向情绪，0代表中性，-1代表负向情绪，-2代表评论内容不涉及该方面。以JSON形式输出结果。'''
print(chat_completion(prompt_str, given_text))

{
  "交通方便": -2,
  "位于商圈附近": -2,
  "是否容易寻找": -2,
  "排队时间": -2,
  "服务人员态度": -2,
  "停车方便": -2,
  "点菜/上菜速度": -2,
  "价格水平": -2,
  "性价比": -2,
  "折扣力度": -2,
  "装修": 0,
  "噪杂情况": -1,
  "就餐空间": -1,
  "卫生情况": -2,
  "菜品分量": 0,
  "菜品口味": 1,
  "菜品卖相": -2,
  "菜品推荐程度": 0
}
